In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    confusion_matrix, 
    classification_report, 
    accuracy_score, 
    roc_auc_score, 
    roc_curve, 
    auc
)

In [2]:
df = pd.read_csv("Raisin_preprocessed.csv")

In [3]:
df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter,Class
0,87524.0,442.246011,253.291155,0.819738,90546.0,0.758651,1184.040,1
1,75166.0,406.690687,243.032436,0.801805,78789.0,0.684130,1121.786,1
2,90856.0,442.267048,266.328318,0.798354,93717.0,0.637613,1208.575,1
3,45928.0,286.540559,208.760042,0.684989,47336.0,0.699599,844.162,1
4,79408.0,352.190770,290.827533,0.590559,81463.0,0.792772,1073.251,1


In [4]:
x = df.drop('Class', axis=1)
y = df[['Class']]

In [5]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

In [9]:
x_scaled_df = pd.DataFrame(
    x_scaled,
    columns=x.columns
)
x_scaled_df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter
0,0.018233,0.114718,-0.008905,0.433793,0.014334,1.182348,0.085566
1,-0.321083,-0.205127,-0.223887,0.209859,-0.299351,-0.340924,-0.154109
2,0.109720,0.114908,0.264303,0.166758,0.098939,-1.291766,0.180025
3,-1.123875,-1.285961,-0.942103,-1.248841,-1.138540,-0.024708,-1.222948
4,-0.204609,-0.695392,0.777711,-2.428005,-0.228007,1.879817,-0.340966


Training Logisic regression with scalling

In [14]:
xtrain, xtest, ytrain, ytest = train_test_split(x_scaled_df, y, test_size=0.2, random_state=42)

In [15]:
xtrain.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter
10,-0.176356,0.179564,-0.448263,0.887128,-0.148044,-0.543162,0.055787
334,-0.394750,-0.855274,0.606989,-2.428005,-0.401912,0.109801,-0.422169
244,-0.030778,-0.441896,0.723825,-1.655140,-0.078888,1.253911,-0.258866
678,2.380226,1.724596,2.458959,0.084868,2.365533,0.659273,1.991428
306,-0.659657,-0.074881,-1.310724,1.324289,-0.683100,1.744358,-0.551551


In [17]:
ytrain.head()

,Class
10,1
334,1
244,1
678,0
306,1


In [36]:
logi = LogisticRegression(max_iter=300)

In [37]:
log_model = logi.fit(xtrain, ytrain)
log_model

c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,300
,multi_class,'deprecated'


In [38]:
print(f'Train Score: {log_model.score(xtrain, ytrain)}')
print(f'Test Score: {log_model.score(xtest, ytest)}')
log_pred = log_model.predict(xtest)
print(f'Confusion Matrix:\n{confusion_matrix(ytest, log_pred)}')
print(f'Classfication Report:\n{classification_report(ytest, log_pred)}')
print(f'Accuracy Score: {accuracy_score(ytest, log_pred)}')

Train Score: 0.8680555555555556
Test Score: 0.8555555555555555
Confusion Matrix:
[[75 11]
 [15 79]]
Classfication Report:
              precision    recall  f1-score   support

           0       0.83      0.87      0.85        86
           1       0.88      0.84      0.86        94

    accuracy                           0.86       180
   macro avg       0.86      0.86      0.86       180
weighted avg       0.86      0.86      0.86       180

Accuracy Score: 0.8555555555555555


Training KNN with scalling

In [ ]:
k_values = range(2,25)
for k in k_values:
    print(f'K Value is: {k}')
    knn = KNeighborsClassifier(n_neighbors=k)
    knn_model = knn.fit(xtrain, ytrain)
    print(f'Train score: {knn_model.score(xtrain, ytrain)}')
    print(f'Test score: {knn_model.score(xtest, ytest)}')
    pred_knn = knn_model.predict(xtest)
    print(f'Confusion matrix:\n{confusion_matrix(ytest, pred_knn)}')
    print(f'Acuuracy Score: {accuracy_score(ytest, pred_knn)}')
    print("-------------------------")

K Value is: 2
Train score: 0.9180555555555555
Test score: 0.8
Confusion matrix:
[[79  7]
 [29 65]]
Acuuracy Score: 0.8
-------------------------
K Value is: 3
Train score: 0.9138888888888889
Test score: 0.8277777777777777
Confusion matrix:
[[72 14]
 [17 77]]
Acuuracy Score: 0.8277777777777777
-------------------------
K Value is: 4
Train score: 0.8944444444444445
Test score: 0.8
Confusion matrix:
[[75 11]
 [25 69]]
Acuuracy Score: 0.8
-------------------------
K Value is: 5


c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classificat

Train score: 0.8847222222222222
Test score: 0.8277777777777777
Confusion matrix:
[[71 15]
 [16 78]]
Acuuracy Score: 0.8277777777777777
-------------------------
K Value is: 6
Train score: 0.8763888888888889
Test score: 0.8277777777777777
Confusion matrix:
[[74 12]
 [19 75]]
Acuuracy Score: 0.8277777777777777
-------------------------
K Value is: 7
Train score: 0.8791666666666667
Test score: 0.8555555555555555
Confusion matrix:
[[72 14]
 [12 82]]
Acuuracy Score: 0.8555555555555555
-------------------------
K Value is: 8
Train score: 0.875
Test score: 0.8222222222222222


c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classificat

Confusion matrix:
[[72 14]
 [18 76]]
Acuuracy Score: 0.8222222222222222
-------------------------
K Value is: 9
Train score: 0.8791666666666667
Test score: 0.85
Confusion matrix:
[[71 15]
 [12 82]]
Acuuracy Score: 0.85
-------------------------
K Value is: 10
Train score: 0.8763888888888889
Test score: 0.8555555555555555
Confusion matrix:
[[73 13]
 [13 81]]


...

Train score: 0.875
Test score: 0.8444444444444444
Confusion matrix:
[[72 14]
 [14 80]]
Acuuracy Score: 0.8444444444444444
-------------------------
K Value is: 14
Train score: 0.8736111111111111
Test score: 0.8444444444444444
Confusion matrix:
[[73 13]
 [15 79]]
Acuuracy Score: 0.8444444444444444
-------------------------
K Value is: 15
Train score: 0.8722222222222222
Test score: 0.85
Confusion matrix:
[[71 15]
 [12 82]]
Acuuracy Score: 0.85
-------------------------
K Value is: 16
Train score: 0.875
Test score: 0.8555555555555555
Confusion matrix:
[[72 14]
 [12 82]]
Acuuracy Score: 0.8555555555555555
-------------------------
K Value is: 17
Train score: 0.8777777777777778
Test score: 0.8444444444444444
Confusion matrix:
[[70 16]
 [12 82]]
Acuuracy Score: 0.8444444444444444
-------------------------
K Value is: 18


c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classificat

Train score: 0.8763888888888889
Test score: 0.85
Confusion matrix:
[[71 15]
 [12 82]]
Acuuracy Score: 0.85
-------------------------
K Value is: 19
Train score: 0.8722222222222222
Test score: 0.85
Confusion matrix:
[[71 15]
 [12 82]]
Acuuracy Score: 0.85
-------------------------
K Value is: 20
Train score: 0.875
Test score: 0.85
Confusion matrix:
[[71 15]
 [12 82]]
Acuuracy Score: 0.85
-------------------------
K Value is: 21
Train score: 0.8736111111111111
Test score: 0.85
Confusion matrix:
[[71 15]
 [12 82]]
Acuuracy Score: 0.85
-------------------------
K Value is: 22
Train score: 0.8722222222222222


c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classificat

Test score: 0.85
Confusion matrix:
[[71 15]
 [12 82]]
Acuuracy Score: 0.85
-------------------------
K Value is: 23
Train score: 0.8736111111111111
Test score: 0.85
Confusion matrix:
[[71 15]
 [12 82]]
Acuuracy Score: 0.85
-------------------------
K Value is: 24
Train score: 0.8694444444444445
Test score: 0.85
Confusion matrix:
[[71 15]
 [12 82]]
Acuuracy Score: 0.85
-------------------------


Training with KNN & Logisic regression without scalling

In [40]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [41]:
xtrain.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter
10,-0.176356,0.179564,-0.448263,0.887128,-0.148044,-0.543162,0.055787
334,-0.394750,-0.855274,0.606989,-2.428005,-0.401912,0.109801,-0.422169
244,-0.030778,-0.441896,0.723825,-1.655140,-0.078888,1.253911,-0.258866
678,2.380226,1.724596,2.458959,0.084868,2.365533,0.659273,1.991428
306,-0.659657,-0.074881,-1.310724,1.324289,-0.683100,1.744358,-0.551551


In [42]:
ytrain.head()

,Class
10,1
334,1
244,1
678,0
306,1


In [43]:
log_model2 = logi.fit(x_train, y_train)

c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [44]:
print(f'Train Score: {log_model2.score(x_train, y_train)}')
print(f'Test Score: {log_model2.score(x_test, y_test)}')
log_pred2 = log_model2.predict(x_test)
print(f'Confusion Matrix:\n{confusion_matrix(y_test, log_pred2)}')
print(f'Classfication Report:\n{classification_report(y_test, log_pred2)}')
print(f'Accuracy Score: {accuracy_score(y_test, log_pred2)}')

Train Score: 0.8666666666666667
Test Score: 0.8611111111111112
Confusion Matrix:
[[76 10]
 [15 79]]
Classfication Report:
              precision    recall  f1-score   support

           0       0.84      0.88      0.86        86
           1       0.89      0.84      0.86        94

    accuracy                           0.86       180
   macro avg       0.86      0.86      0.86       180
weighted avg       0.86      0.86      0.86       180

Accuracy Score: 0.8611111111111112


In [45]:
k_values = range(2,25)
for k in k_values:
    print(f'K Value is: {k}')
    knn2 = KNeighborsClassifier(n_neighbors=k)
    knn_model2 = knn2.fit(x_train, y_train)
    print(f'Train score: {knn_model2.score(x_train, y_train)}')
    print(f'Test score: {knn_model2.score(x_test, y_test)}')
    pred_knn2 = knn_model2.predict(x_test)
    print(f'Confusion matrix:\n{confusion_matrix(y_test, pred_knn2)}')
    print(f'Acuuracy Score: {accuracy_score(y_test, pred_knn2)}')
    print("-------------------------")

K Value is: 2
Train score: 0.8972222222222223
Test score: 0.7333333333333333
Confusion matrix:
[[78  8]
 [40 54]]
Acuuracy Score: 0.7333333333333333
-------------------------
K Value is: 3
Train score: 0.8958333333333334
Test score: 0.8222222222222222
Confusion matrix:
[[71 15]
 [17 77]]
Acuuracy Score: 0.8222222222222222
-------------------------
K Value is: 4


c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Train score: 0.875
Test score: 0.8166666666666667
Confusion matrix:
[[75 11]
 [22 72]]
Acuuracy Score: 0.8166666666666667
-------------------------
K Value is: 5
Train score: 0.8763888888888889
Test score: 0.8444444444444444
Confusion matrix:
[[71 15]
 [13 81]]
Acuuracy Score: 0.8444444444444444
-------------------------
K Value is: 6


c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classificat

Train score: 0.8680555555555556
Test score: 0.8388888888888889
Confusion matrix:
[[75 11]
 [18 76]]
Acuuracy Score: 0.8388888888888889
-------------------------
K Value is: 7
Train score: 0.8694444444444445
Test score: 0.85
Confusion matrix:
[[72 14]
 [13 81]]
Acuuracy Score: 0.85
-------------------------
K Value is: 8
Train score: 0.8666666666666667
Test score: 0.85
Confusion matrix:
[[72 14]
 [13 81]]
Acuuracy Score: 0.85
-------------------------
K Value is: 9
Train score: 0.8583333333333333
Test score: 0.85
Confusion matrix:
[[70 16]
 [11 83]]
Acuuracy Score: 0.85
-------------------------
K Value is: 10
Train score: 0.8638888888888889
Test score: 0.8555555555555555


...

c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classificat

Train score: 0.85
Test score: 0.8611111111111112
Confusion matrix:
[[71 15]
 [10 84]]
Acuuracy Score: 0.8611111111111112
-------------------------
K Value is: 17
Train score: 0.8416666666666667
Test score: 0.8444444444444444
Confusion matrix:
[[68 18]
 [10 84]]
Acuuracy Score: 0.8444444444444444
-------------------------
K Value is: 18
Train score: 0.8486111111111111
Test score: 0.8555555555555555
Confusion matrix:
[[70 16]
 [10 84]]
Acuuracy Score: 0.8555555555555555
-------------------------
K Value is: 19
Train score: 0.8347222222222223
Test score: 0.8555555555555555
Confusion matrix:
[[69 17]
 [ 9 85]]
Acuuracy Score: 0.8555555555555555
-------------------------
K Value is: 20


c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classificat

Train score: 0.8375
Test score: 0.8611111111111112
Confusion matrix:
[[70 16]
 [ 9 85]]
Acuuracy Score: 0.8611111111111112
-------------------------
K Value is: 21
Train score: 0.8319444444444445
Test score: 0.8555555555555555
Confusion matrix:
[[69 17]
 [ 9 85]]
Acuuracy Score: 0.8555555555555555
-------------------------
K Value is: 22
Train score: 0.8375
Test score: 0.8555555555555555
Confusion matrix:
[[70 16]
 [10 84]]
Acuuracy Score: 0.8555555555555555
-------------------------
K Value is: 23
Train score: 0.8305555555555556
Test score: 0.8611111111111112
Confusion matrix:
[[69 17]
 [ 8 86]]
Acuuracy Score: 0.8611111111111112
-------------------------
K Value is: 24
Train score: 0.8333333333333334
Test score: 0.8555555555555555
Confusion matrix:
[[69 17]
 [ 9 85]]
Acuuracy Score: 0.8555555555555555
-------------------------


c:\Users\MD Maruf\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
